In [95]:
from gensim.utils import tokenize
import pandas as pd
import numpy as np
import sys
import os
import liwc_utils
from collections import defaultdict
df = pd.read_csv("/Users/Lara/Dropbox/LanguageGender/Data/Advocates/Cleaned/cleaned_Advocates_Full_Apr13.csv")
liwc_cats = ['Incl', 'Article', 'Inhib', 'You', 'Present', 'Social', 'We', 'Feel', 'Ingest', 'Time', 'Cause', 'Leisure', 'Quant', 'Money', 'Anx', 'Assent', 'Conj', 'Family', 'Negate', 'CogMech', 'Achiev', 'See',
    'Relig', 'Certain', 'Ipron', 'Tentat', 'Hear', 'Percept', 'AuxVb', 'Nonflu', 'Pronoun', 'Relativ', 'Bio', 'Friends', 'Work', 'I', 'Negemo', 'Anger', 'Body', 'SheHe', 'Ppron', 'Affect', 'Health', 'Discrep', 'Motion',
    'Sad', 'Posemo', 'Humans', 'Home', 'Swear', 'They', 'Future', 'Excl', 'Filler', 'Space', 'Death', 'Prep', 'Past', 'Sexual', 'Insight', 'Funct', 'Adverbs', 'Numbers', 'Verbs']


In [96]:
def compute_avg(col):
    female = defaultdict(int)
    male = defaultdict(int)
    num_female = 0
    num_male = 0
    liwc_df = []
    for index, row in df.iterrows():
        l_dict = liwc_utils.response_to_liwc(row[col])
        liwc_row = [row['ResponseId']]
        for c in liwc_cats:
            liwc_row.append(l_dict.get(c, 0))
        liwc_df.append(liwc_row)
        
        if row['Gender'] == 'Male':
            for cat, val in l_dict.items():
                male[cat] += val
            num_male += 1
        elif row['Gender'] == 'Female':
            for cat, val in l_dict.items():
                female[cat] += val
            num_female += 1

    for cat, val in female.copy().items():
        female[cat] = val / num_female
    for cat, val in male.copy().items():
        male[cat] = val / num_male

    liwc_df = pd.DataFrame(liwc_df, columns=['ResponseId']+liwc_cats)
    return female, male, liwc_df
        

In [97]:
female_q1, male_q1, liwc_df = compute_avg("Intro.Question.1")
new_df = pd.merge(df, liwc_df)

female_q2, male_q2, liwc_df = compute_avg("Intro.Question.2")
new_df = pd.merge(new_df, liwc_df, on=['ResponseId'], suffixes=['_q1', '_q2'])

female_exp, male_exp, liwc_df = compute_avg("text")
new_df = pd.merge(new_df, liwc_df, on=['ResponseId'])


In [98]:
new_df.rename(columns={c:c+"_exp" for c in liwc_cats}, inplace=True)

In [99]:
rows = []
for c in liwc_cats:
    rows.append([female_q1[c], male_q1[c], female_q2[c], male_q2[c], female_exp[c], male_exp[c]])

In [100]:
liwc_avgs = pd.DataFrame(rows, index=liwc_cats, columns=['Female_Q1', 'Male_Q1', 'Female_Q2', 'Male_Q2', 'Female_Exp', 'Male_Exp'])

In [101]:
liwc_avgs['q1_diff'] = liwc_avgs['Female_Q1'] - liwc_avgs['Male_Q1']
liwc_avgs['q2_diff'] = liwc_avgs['Female_Q2'] - liwc_avgs['Male_Q2']
liwc_avgs['exp_diff'] = liwc_avgs['Female_Exp'] - liwc_avgs['Male_Exp']


In [102]:
liwc_avgs

,Female_Q1,Male_Q1,Female_Q2,Male_Q2,Female_Exp,Male_Exp,q1_diff,q2_diff,exp_diff
Incl,0.907692,0.980149,1.141026,1.057072,2.510256,2.203474,-0.072457,0.083954,0.306782
Article,0.487179,0.689826,2.166667,2.114144,1.266667,1.488834,-0.202647,0.052523,-0.222167
Inhib,0.174359,0.173697,0.128205,0.163772,0.212821,0.258065,0.000662,-0.035567,-0.045244
You,0.964103,0.945409,0.812821,0.669975,0.730769,0.657568,0.018693,0.142845,0.073201
Present,1.879487,1.903226,1.820513,1.578164,2.756410,2.481390,-0.023739,0.242349,0.275021
...,...,...,...,...,...,...,...,...,...
Insight,0.828205,0.813896,0.712821,0.580645,0.679487,0.615385,0.014309,0.132175,0.064103
Funct,8.489744,9.178660,13.146154,12.315136,15.851282,15.545906,-0.688916,0.831017,0.305376
Adverbs,0.341026,0.272953,0.553846,0.583127,0.735897,0.736973,0.068073,-0.029280,-0.001075
Numbers,0.028205,0.022333,0.066667,0.069479,0.056410,0.059553,0.005873,-0.002812,-0.003143


In [103]:
liwc_avgs[['Female_Q1', 'Male_Q1', 'q1_diff']].sort_values(by='q1_diff')


,Female_Q1,Male_Q1,q1_diff
Funct,8.489744,9.178660,-0.688916
Prep,2.184615,2.429280,-0.244665
Article,0.487179,0.689826,-0.202647
AuxVb,1.374359,1.508685,-0.134326
Pronoun,2.566667,2.684864,-0.118197
...,...,...,...
Cause,0.320513,0.290323,0.030190
I,0.248718,0.213400,0.035318
Tentat,0.469231,0.421836,0.047395
Adverbs,0.341026,0.272953,0.068073


In [104]:
liwc_avgs[['Female_Q2', 'Male_Q2', 'q2_diff']].sort_values(by='q2_diff')


,Female_Q2,Male_Q2,q2_diff
Prep,2.997436,3.183623,-0.186187
Relativ,2.405128,2.526055,-0.120926
Space,1.023077,1.096774,-0.073697
Sad,0.100000,0.153846,-0.053846
Motion,0.492308,0.531017,-0.038710
...,...,...,...
Verbs,3.202564,2.803970,0.398594
Pronoun,3.453846,3.014888,0.438958
Social,2.641026,2.198511,0.442514
CogMech,5.017949,4.538462,0.479487


In [108]:
liwc_avgs[['Female_Exp', 'Male_Exp', 'exp_diff']].sort_values(by='exp_diff')


,Female_Exp,Male_Exp,exp_diff
Article,1.266667,1.488834,-0.222167
Work,1.258974,1.411911,-0.152936
Affect,1.882051,2.029777,-0.147725
Space,1.107692,1.238213,-0.130521
Relativ,2.466667,2.585608,-0.118941
...,...,...,...
Incl,2.510256,2.203474,0.306782
CogMech,5.869231,5.555831,0.313400
Social,3.776923,3.459057,0.317866
Ppron,2.907692,2.570720,0.336973


In [113]:
new_df = new_df.drop(columns=['Unnamed: 0'])

,ResponseId,Intro.Question.1,Intro.Question.2,Education,Age,Gender,Gender_3_TEXT,Attention.Check,Ethnicity,Ethnicity_6_TEXT,...,Space_exp,Death_exp,Prep_exp,Past_exp,Sexual_exp,Insight_exp,Funct_exp,Adverbs_exp,Numbers_exp,Verbs_exp
0,R_9Y9wQmmp4cVpBBL,Accepting responsibility for quality service,"Taking ownership of a customers issues, and em...",Some college,56,Male,NaN,Purple,White or Caucasian,NaN,...,1,0,3,0,0,0,9,0,0,2
1,R_Cl7x1KGNXWuipwd,you should be responsible for actions taken,sales of an item,Bachelor degree,48,Male,NaN,Purple,Black of African American,NaN,...,0,0,0,0,0,0,0,0,0,0
2,R_tGM6VkVXJS0XJ2F,"Being responsible, both positive and negative,...",Taking ownership of a failure.,Graduate degree,36,Male,NaN,Purple,White or Caucasian,NaN,...,2,0,4,0,0,0,11,1,0,2
3,R_10u6pCvlekYLVZ7,Being the responsibility of upholding a set st...,"Aligning ones words with action. For instance,...",Bachelor degree,37,Male,NaN,Purple,Latino or Latina,NaN,...,1,0,5,0,0,0,14,0,0,4
4,R_27CLv5B1e26pqOW,"Responsibility, taking measured action and own...",Making a commitment to have a difficult conver...,Graduate degree,35,Female,NaN,Purple,White or Caucasian,NaN,...,0,0,2,0,0,0,8,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
793,R_1iao1kzm1vuBYvp,Accountability means giving accounts and recor...,An employee is responsible for completing a ta...,Bachelor degree,58,Female,NaN,Purple,White or Caucasian,NaN,...,0,0,2,0,0,1,10,0,1,3
794,R_RUqCH18mSclDcdj,Accountability is taking responsibility for yo...,If you go to the bank to make a deposit and lo...,Bachelor degree,39,Female,NaN,Purple,White or Caucasian,NaN,...,0,0,3,0,0,0,22,0,0,3
795,R_6xPrjD9SMmppz3P,"I think, accountability means being responsibl...","In this case, an example of an action that ref...",Graduate degree,38,Female,NaN,Purple,White or Caucasian,NaN,...,4,0,9,0,0,1,36,2,0,9
796,R_23UHo6ZUPWcrqjE,responsibility for outcomes,If an error happens that you are responsible f...,Some college,49,Female,NaN,Purple,White or Caucasian,NaN,...,0,0,0,0,0,0,1,0,0,0


In [114]:
new_df.to_csv("/Users/Lara/Dropbox/LanguageGender/Data/Advocates/Cleaned/liwc_accountability.csv", index=False)